In [1]:
import pandas as pd
import numpy as np
import datetime
import copy

data = pd.read_excel('last Ю3 (1).xlsx', sheet_name='ГРП')
dummy = pd.read_excel('last Ю3 (1).xlsx', sheet_name='МЭР')
dummy.head(20)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт
0,2005-04-01,1,NaN,0.0,NaN,NaN,NaN,NaN
1,2005-05-01,1,NaN,0.0,NaN,NaN,NaN,NaN
2,2005-06-01,1,NaN,0.0,NaN,NaN,NaN,NaN
3,2005-07-01,1,NaN,0.0,NaN,NaN,NaN,NaN
4,2005-08-01,1,NaN,0.0,NaN,NaN,NaN,NaN
5,2005-09-01,1,NaN,0.0,NaN,NaN,NaN,NaN
6,2005-10-01,1,NaN,0.0,NaN,NaN,NaN,NaN
7,2005-11-01,1,NaN,0.0,NaN,NaN,NaN,NaN
8,2005-12-01,1,NaN,0.0,NaN,NaN,NaN,NaN
9,2006-01-01,1,NaN,0.0,NaN,NaN,NaN,NaN


In [2]:
date_column_name = 'Дата ВНР после'
target = 'Скважина №'
data['Скважина №'] = data.apply(
            lambda x:
            str(x['Скважина №']), axis=1)
dummy['Скважина №'] = dummy.apply(
            lambda x:
            str(x['Скважина №']), axis=1)
print(data.shape, dummy.shape, date_column_name)

(852, 11) (315120, 8) Дата ВНР после


In [3]:
dummy['Index'] = dummy.index
dummy1 = dummy.drop(dummy[dummy['Пластовое давление (ТР), атм'] != 0].index)
dummy1 = dummy1.drop(dummy1[dummy1['Забойное давление (ТР), атм'] != 0].index)
dummy1 = dummy1.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy1['кол'] = dummy1.apply(lambda x: len(x['Index']), axis=1)
dummy1 = dummy1.drop(dummy1[dummy1['кол'] < 12].index)

delete = []
for i in dummy1['Index']:
    delete += i
dummy = dummy[~dummy['Index'].isin(delete)].reset_index(drop=True)

dummy2 = dummy[dummy['Забойное давление (ТР), атм'].isna()]
dummy2 = dummy2.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy2['забойки'] = dummy2.apply(lambda x: len(x['Index']), axis=1)
dummy2 = dummy2.drop(dummy2[dummy2['забойки'] < 12].index)
            # wells_del_both=dummy1['Index'].tolist()
delete_wellbore = []
for i in dummy2['Index']:
    delete_wellbore += i
dummy = dummy[~dummy['Index'].isin(delete_wellbore)].reset_index(drop=True)

dummy = dummy.drop(dummy[dummy['Дебит нефти, т/сут'] == 0].index)
            #n = 1
            #z = dummy['Пласт'].value_counts()[:n].index.tolist()
            #plast = str(z[0])
plast = 'Ю3'
dummy = dummy.drop(dummy[dummy['Пласт'] != plast].index).reset_index(
                drop=True)
dummy_res_zero = dummy.drop(dummy[dummy['Пластовое давление (ТР), атм'] != 0].index)
dummy_res_zero = dummy_res_zero.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy_res_zero['нули_пласт'] = dummy_res_zero.apply(lambda x: len(x['Index']), axis=1)
dummy_res_zero = dummy_res_zero.drop(dummy_res_zero[dummy_res_zero['нули_пласт'] < 12].index)
            # wells_del_both=dummy1['Index'].tolist()
delete_reservoir = []
for i in dummy_res_zero['Index']:
        delete_reservoir += i
dummy = dummy[~dummy['Index'].isin(delete_reservoir)].reset_index(drop=True)

dummy_bore_zero = dummy.drop(dummy[dummy['Забойное давление (ТР), атм'] != 0].index)
dummy_bore_zero = dummy_bore_zero.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy_bore_zero['нули_заб'] = dummy_bore_zero.apply(lambda x: len(x['Index']), axis=1)
dummy_bore_zero = dummy_bore_zero.drop(dummy_bore_zero[dummy_bore_zero['нули_заб'] < 12].index)
            # wells_del_both=dummy1['Index'].tolist()
delete_bore = []
for i in dummy_bore_zero['Index']:
    delete_bore += i
dummy = dummy[~dummy['Index'].isin(delete_bore)].reset_index(drop=True)

dummy_40 = dummy[dummy['Пластовое давление (ТР), атм'].isna()]
dummy_40 = dummy_40.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy_40['na_пласт'] = dummy_40.apply(lambda x: len(x['Index']), axis=1)
dummy_40 = dummy_40.drop(dummy_40[dummy_40['na_пласт'] < 12].index)
            # wells_del_both=dummy1['Index'].tolist()
del_plast = []
for i in dummy_40['Index']:
    del_plast += i
dummy = dummy[~dummy['Index'].isin(del_plast)].reset_index(drop=True)
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-03-01,1,84.09,58.50,20.86,0.0,196.4,Ю3,131
1,2016-04-01,1,71.61,47.27,24.90,0.0,172.5,Ю3,132
2,2016-05-01,1,84.65,61.39,17.50,0.0,161.6,Ю3,133
3,2016-06-01,1,79.36,60.95,12.62,0.0,149.5,Ю3,134
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
...,...,...,...,...,...,...,...,...,...
22868,2019-11-01,99,62.63,7.13,87.04,0.0,83.8,Ю3,314891
22869,2019-12-01,99,53.38,6.24,86.69,0.0,74.3,Ю3,314892
22870,2020-01-01,99,38.98,5.24,84.70,0.0,82.5,Ю3,314893
22871,2020-02-01,99,38.48,5.55,83.60,0.0,61.2,Ю3,314894


In [4]:
dummy1 = dummy.groupby(target).agg(
                {'Дата': lambda x:
                x.tolist()}
            )
dummy1['История'] = dummy1.apply(
                lambda x:
                1 if len(x['Дата']) >= 6
                else 0, axis=1)
dummy1 = dummy1.drop(dummy1[dummy1['История'] != 1].index)
dummy1.reset_index(inplace=True)
a = dummy1['Скважина №'].tolist()
dummy = dummy[dummy['Скважина №'].isin(a)]
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-03-01,1,84.09,58.50,20.86,0.0,196.4,Ю3,131
1,2016-04-01,1,71.61,47.27,24.90,0.0,172.5,Ю3,132
2,2016-05-01,1,84.65,61.39,17.50,0.0,161.6,Ю3,133
3,2016-06-01,1,79.36,60.95,12.62,0.0,149.5,Ю3,134
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
...,...,...,...,...,...,...,...,...,...
22863,2019-09-01,96,114.20,4.17,95.85,0.0,72.9,Ю3,308223
22864,2019-10-01,96,97.35,2.41,97.19,0.0,71.8,Ю3,308224
22865,2019-11-01,96,94.11,2.59,96.87,0.0,65.5,Ю3,308225
22866,2019-12-01,96,90.58,1.65,97.93,0.0,51.8,Ю3,308226


In [5]:
dummy["Пластовое давление (ТР), атм"] = dummy["Пластовое давление (ТР), атм"].replace(0, np.nan).bfill()
dummy["Забойное давление (ТР), атм"] = dummy["Забойное давление (ТР), атм"].replace(0, np.nan).bfill()
dummy['депрессия'] = dummy.apply(
                lambda x:
                x['Пластовое давление (ТР), атм'] - x['Забойное давление (ТР), атм'], axis=1)
dummy = dummy.drop(dummy[dummy['депрессия'] <= 0].index)
dummy = dummy.drop('депрессия', axis=1)
dummy

C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-03-01,1,84.09,58.50,20.86,263.0,196.4,Ю3,131
1,2016-04-01,1,71.61,47.27,24.90,263.0,172.5,Ю3,132
2,2016-05-01,1,84.65,61.39,17.50,263.0,161.6,Ю3,133
3,2016-06-01,1,79.36,60.95,12.62,263.0,149.5,Ю3,134
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
...,...,...,...,...,...,...,...,...,...
22862,2019-08-01,96,136.02,3.15,97.36,266.0,55.1,Ю3,308222
22863,2019-09-01,96,114.20,4.17,95.85,266.0,72.9,Ю3,308223
22864,2019-10-01,96,97.35,2.41,97.19,266.0,71.8,Ю3,308224
22865,2019-11-01,96,94.11,2.59,96.87,266.0,65.5,Ю3,308225


In [6]:
dummy.loc[dummy['Скважина №'] == '110']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
212,2020-02-01,110,120.45,28.34,73.24,258.0,69.6,Ю3,9268
213,2020-03-01,110,84.35,31.32,57.76,258.0,69.6,Ю3,9269
214,2020-04-01,110,25.53,20.42,9.00,258.0,71.1,Ю3,9270
215,2020-05-01,110,15.08,12.08,8.85,258.0,42.5,Ю3,9271
216,2020-06-01,110,17.41,12.67,17.17,258.0,63.5,Ю3,9272
217,2020-07-01,110,17.46,11.88,22.57,258.0,60.9,Ю3,9273
218,2020-08-01,110,16.66,11.78,19.56,258.0,53.3,Ю3,9274
219,2020-09-01,110,16.96,10.76,27.82,258.0,52.3,Ю3,9275
220,2020-10-01,110,16.71,11.46,21.98,208.0,47.2,Ю3,9276
221,2020-11-01,110,15.69,10.72,22.32,208.0,61.9,Ю3,9277


In [7]:
data.loc[data['Скважина №'] == '110']

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт
14,14,110,817.866126,0.0,0,0,0,2020-02-17,ГС,ВНС,Ю3


In [8]:
data.loc[data['Скважина №'] == '1004_ГРП']

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт
3,3,1004_ГРП,0.0,1.0,100,4,100,2011-02-04,ННС+ГРП,ГРП,Ю3


In [9]:
#работаем с Л
data_L=copy.deepcopy(data)
data_L['удалю'] = data_L.apply(
            lambda x:
            'нет' if 'Л' in str(x['Скважина №'])
            else 'да', axis=1)
data_L = data_L.drop(data_L[data_L['удалю'] == 'да'].index).reset_index(drop=True)
data_L = data_L.drop(['удалю'], axis=1)

data_L

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт
0,139,1556Л_ГРП,0.000000,1.0,100,4,100,2018-10-05,ННС+ГРП,ГРП,Ю3
1,432,527Л_ГРП,534.756954,3.0,100,4,100,2014-03-04,ГС+ГРП,ГРП,Ю3
2,669,123Л,313.336879,0.0,0,0,0,2009-01-20,ГС,ВНС,Ю3
3,670,123Л_ГРП,313.336879,1.0,100,4,100,2009-03-16,ГС+ГРП,ГРП,Ю3
4,671,123Л_ФРАК_1,313.336879,1.0,100,4,100,2012-07-15,ГС+ГРП,ГРП,Ю3
...,...,...,...,...,...,...,...,...,...,...,...
116,844,85Л_ФРАК_3,439.829512,1.0,100,4,100,2016-10-11,ГС+ГРП,ГРП,Ю3
117,845,87Л_ГРП,343.011662,1.0,100,4,100,2011-05-24,ГС+ГРП,ГРП,Ю3
118,847,897Л_ГРП,0.000000,1.0,100,4,100,2011-01-17,ННС+ГРП,ГРП,Ю3
119,849,906Л_ГРП,0.000000,1.0,100,4,100,2010-11-26,ННС+ГРП,ГРП,Ю3


In [10]:
data_L['для_грп'] = data_L.apply(
                lambda x:
                x['Скважина №'].partition('_')[0] if '_' in str(x['Скважина №'])
                else x['Скважина №'], axis=1)
data_L

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт,для_грп
0,139,1556Л_ГРП,0.000000,1.0,100,4,100,2018-10-05,ННС+ГРП,ГРП,Ю3,1556Л
1,432,527Л_ГРП,534.756954,3.0,100,4,100,2014-03-04,ГС+ГРП,ГРП,Ю3,527Л
2,669,123Л,313.336879,0.0,0,0,0,2009-01-20,ГС,ВНС,Ю3,123Л
3,670,123Л_ГРП,313.336879,1.0,100,4,100,2009-03-16,ГС+ГРП,ГРП,Ю3,123Л
4,671,123Л_ФРАК_1,313.336879,1.0,100,4,100,2012-07-15,ГС+ГРП,ГРП,Ю3,123Л
...,...,...,...,...,...,...,...,...,...,...,...,...
116,844,85Л_ФРАК_3,439.829512,1.0,100,4,100,2016-10-11,ГС+ГРП,ГРП,Ю3,85Л
117,845,87Л_ГРП,343.011662,1.0,100,4,100,2011-05-24,ГС+ГРП,ГРП,Ю3,87Л
118,847,897Л_ГРП,0.000000,1.0,100,4,100,2011-01-17,ННС+ГРП,ГРП,Ю3,897Л
119,849,906Л_ГРП,0.000000,1.0,100,4,100,2010-11-26,ННС+ГРП,ГРП,Ю3,906Л


In [11]:

data_L = data_L.groupby('для_грп').agg(
                {date_column_name: lambda x:
                x.tolist()[-1] })
data_L

,Дата ВНР после
для_грп,
1230Л,2013-09-09
123Л,2012-07-15
1311Л,2010-05-15
133Л,2013-10-11
142Л,2013-04-25
...,...
87Л,2011-05-24
897Л,2011-01-17
8Л,2011-07-03


In [12]:
data_L.reset_index(inplace=True)
data_L

,для_грп,Дата ВНР после
0,1230Л,2013-09-09
1,123Л,2012-07-15
2,1311Л,2010-05-15
3,133Л,2013-10-11
4,142Л,2013-04-25
...,...,...
67,87Л,2011-05-24
68,897Л,2011-01-17
69,8Л,2011-07-03
70,906Л,2010-11-26


In [13]:
wells_L = data_L['для_грп'].tolist()
data_L['Скважина №'] = wells_L
data_L = data_L.drop(['для_грп'], axis=1)
data_L

,Дата ВНР после,Скважина №
0,2013-09-09,1230Л
1,2012-07-15,123Л
2,2010-05-15,1311Л
3,2013-10-11,133Л
4,2013-04-25,142Л
...,...,...
67,2011-05-24,87Л
68,2011-01-17,897Л
69,2011-07-03,8Л
70,2010-11-26,906Л


In [14]:
data_L['Скважина №'] = data_L.apply(
            lambda x:
            str(x['Скважина №']).replace('Л', '') ,
            axis=1
            )
data_L    

,Дата ВНР после,Скважина №
0,2013-09-09,1230
1,2012-07-15,123
2,2010-05-15,1311
3,2013-10-11,133
4,2013-04-25,142
...,...,...
67,2011-05-24,87
68,2011-01-17,897
69,2011-07-03,8
70,2010-11-26,906


In [15]:
dummy = dummy.merge(data_L, on=target, how='left')
dummy.head(10)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
0,2016-03-01,1,84.09,58.50,20.86,263.0,196.4,Ю3,131,NaT
1,2016-04-01,1,71.61,47.27,24.90,263.0,172.5,Ю3,132,NaT
2,2016-05-01,1,84.65,61.39,17.50,263.0,161.6,Ю3,133,NaT
3,2016-06-01,1,79.36,60.95,12.62,263.0,149.5,Ю3,134,NaT
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135,NaT
5,2016-08-01,1,84.28,59.86,19.19,263.0,118.3,Ю3,136,NaT
6,2016-09-01,1,85.31,68.98,8.01,263.0,107.1,Ю3,137,NaT
7,2016-10-01,1,88.38,71.09,8.49,263.0,94.8,Ю3,138,NaT
8,2016-11-01,1,78.99,62.62,9.81,263.0,79.4,Ю3,139,NaT
9,2016-12-01,1,68.36,53.63,10.74,263.0,69.8,Ю3,140,NaT


In [16]:
dummy.loc[dummy['Скважина №'] == '123']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
685,2019-10-01,123,94.25,32.39,60.90,212.0,102.6,Ю3,20980,2012-07-15
686,2020-02-01,123,72.77,28.63,55.23,212.0,83.3,Ю3,20984,2012-07-15
687,2020-03-01,123,72.70,28.73,55.04,212.0,80.0,Ю3,20985,2012-07-15
688,2020-04-01,123,70.98,27.92,55.24,212.0,83.6,Ю3,20986,2012-07-15
689,2020-05-01,123,67.34,25.05,57.67,212.0,83.1,Ю3,20987,2012-07-15
690,2020-06-01,123,62.49,21.67,60.54,212.0,77.0,Ю3,20988,2012-07-15
691,2020-07-01,123,63.44,21.37,61.67,182.0,74.7,Ю3,20989,2012-07-15
692,2020-08-01,123,63.13,22.03,60.31,182.0,74.9,Ю3,20990,2012-07-15
693,2020-09-01,123,65.45,19.73,65.71,182.0,74.0,Ю3,20991,2012-07-15
694,2020-10-01,123,69.91,19.87,67.67,182.0,75.2,Ю3,20992,2012-07-15


In [17]:
if dummy.shape[0] != 0:
    dummy[target] = dummy.apply(
    lambda x:
    str(x[target])
    + ('Л' if x['Дата'] <= x[date_column_name] else ''),
    axis=1
                )
dummy.loc[dummy['Скважина №'] == '123Л']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после


In [18]:
dummy.loc[dummy['Скважина №'] == '897Л']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после


In [19]:
dummy = dummy.drop(date_column_name, axis=1)
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-03-01,1,84.09,58.50,20.86,263.0,196.4,Ю3,131
1,2016-04-01,1,71.61,47.27,24.90,263.0,172.5,Ю3,132
2,2016-05-01,1,84.65,61.39,17.50,263.0,161.6,Ю3,133
3,2016-06-01,1,79.36,60.95,12.62,263.0,149.5,Ю3,134
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
...,...,...,...,...,...,...,...,...,...
22506,2019-08-01,96,136.02,3.15,97.36,266.0,55.1,Ю3,308222
22507,2019-09-01,96,114.20,4.17,95.85,266.0,72.9,Ю3,308223
22508,2019-10-01,96,97.35,2.41,97.19,266.0,71.8,Ю3,308224
22509,2019-11-01,96,94.11,2.59,96.87,266.0,65.5,Ю3,308225


In [20]:
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-03-01,1,84.09,58.50,20.86,263.0,196.4,Ю3,131
1,2016-04-01,1,71.61,47.27,24.90,263.0,172.5,Ю3,132
2,2016-05-01,1,84.65,61.39,17.50,263.0,161.6,Ю3,133
3,2016-06-01,1,79.36,60.95,12.62,263.0,149.5,Ю3,134
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
...,...,...,...,...,...,...,...,...,...
22506,2019-08-01,96,136.02,3.15,97.36,266.0,55.1,Ю3,308222
22507,2019-09-01,96,114.20,4.17,95.85,266.0,72.9,Ю3,308223
22508,2019-10-01,96,97.35,2.41,97.19,266.0,71.8,Ю3,308224
22509,2019-11-01,96,94.11,2.59,96.87,266.0,65.5,Ю3,308225


In [21]:
data['для_грп'] = data.apply(
                lambda x:
                x['Скважина №'].partition('_')[0] if '_' in str(x['Скважина №'])
                else x['Скважина №'], axis=1)
data

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт,для_грп
0,0,1_ГРП,0.000000,1.0,100,4,100,2016-03-12,ННС+ГРП,ГРП,Ю3,1
1,1,10,0.000000,0.0,0,0,0,2008-10-12,ННС,ВНС,Ю3,10
2,2,10_ГРП,0.000000,1.0,100,4,100,2013-03-03,ННС+ГРП,ГРП,Ю3,10
3,3,1004_ГРП,0.000000,1.0,100,4,100,2011-02-04,ННС+ГРП,ГРП,Ю3,1004
4,4,1004_ФРАК_1,0.000000,1.0,100,4,100,2013-01-31,ННС+ГРП,ГРП,Ю3,1004
...,...,...,...,...,...,...,...,...,...,...,...,...
847,847,897Л_ГРП,0.000000,1.0,100,4,100,2011-01-17,ННС+ГРП,ГРП,Ю3,897Л
848,848,897_ГРП,0.000000,1.0,100,4,100,2019-05-11,ННС+ГРП,ГРП,Ю3,897
849,849,906Л_ГРП,0.000000,1.0,100,4,100,2010-11-26,ННС+ГРП,ГРП,Ю3,906Л
850,850,909Л_ГРП,294.803324,1.0,100,4,100,2011-01-10,ГС+ГРП,ГРП,Ю3,909Л


In [22]:
#важно
data['fix'] = data.apply(
                lambda x:
                1 if 'ГРП' in str(x['Скважина №'])
                else 0, axis=1)
data = data.drop(data[data['fix'] == 0].index)
data.head()

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт,для_грп,fix
0,0,1_ГРП,0.0,1.0,100,4,100,2016-03-12,ННС+ГРП,ГРП,Ю3,1,1
2,2,10_ГРП,0.0,1.0,100,4,100,2013-03-03,ННС+ГРП,ГРП,Ю3,10,1
3,3,1004_ГРП,0.0,1.0,100,4,100,2011-02-04,ННС+ГРП,ГРП,Ю3,1004,1
6,6,1005_ГРП,0.0,1.0,100,4,100,2011-08-31,ННС+ГРП,ГРП,Ю3,1005,1
7,7,1006_ГРП,0.0,1.0,100,4,100,2019-08-16,ННС+ГРП,ГРП,Ю3,1006,1


In [23]:
data

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт,для_грп,fix
0,0,1_ГРП,0.000000,1.0,100,4,100,2016-03-12,ННС+ГРП,ГРП,Ю3,1,1
2,2,10_ГРП,0.000000,1.0,100,4,100,2013-03-03,ННС+ГРП,ГРП,Ю3,10,1
3,3,1004_ГРП,0.000000,1.0,100,4,100,2011-02-04,ННС+ГРП,ГРП,Ю3,1004,1
6,6,1005_ГРП,0.000000,1.0,100,4,100,2011-08-31,ННС+ГРП,ГРП,Ю3,1005,1
7,7,1006_ГРП,0.000000,1.0,100,4,100,2019-08-16,ННС+ГРП,ГРП,Ю3,1006,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,847,897Л_ГРП,0.000000,1.0,100,4,100,2011-01-17,ННС+ГРП,ГРП,Ю3,897Л,1
848,848,897_ГРП,0.000000,1.0,100,4,100,2019-05-11,ННС+ГРП,ГРП,Ю3,897,1
849,849,906Л_ГРП,0.000000,1.0,100,4,100,2010-11-26,ННС+ГРП,ГРП,Ю3,906Л,1
850,850,909Л_ГРП,294.803324,1.0,100,4,100,2011-01-10,ГС+ГРП,ГРП,Ю3,909Л,1


In [24]:
no_grp = data.groupby('для_грп').agg(
                {'ГС/ННС': lambda x:
                x.tolist()}
            )
no_grp.reset_index(inplace=True)
no_grp.loc[no_grp['для_грп'] == '1004']

,для_грп,ГС/ННС
2,1004,[ННС+ГРП]


In [25]:
no_grp['ГРП?'] = no_grp.apply(
                lambda x:
                0 if 'ГРП' not in str(x['ГС/ННС'])
                else 1, axis=1)
no_grp.loc[no_grp['для_грп'] == '1004']

,для_грп,ГС/ННС,ГРП?
2,1004,[ННС+ГРП],1


In [26]:
no_grp = no_grp.drop(no_grp[no_grp['ГРП?'] == 1].index)
no_grp.loc[no_grp['для_грп'] == '110']

,для_грп,ГС/ННС,ГРП?


In [27]:
spisok = no_grp['для_грп'].tolist()
'110' in spisok

False

In [28]:
no_grp

,для_грп,ГС/ННС,ГРП?


In [29]:
step = data.groupby('для_грп').agg(
                {date_column_name: lambda x:
                x.tolist()[0] })

step

,Дата ВНР после
для_грп,
1,2016-03-12
10,2013-03-03
1004,2011-02-04
1005,2011-08-31
1006,2019-08-16
...,...
936,2008-09-27
942,2013-06-19
96,2019-07-31


In [30]:
step.reset_index(inplace=True)

In [31]:
step.loc[step['для_грп'] == '1004']

,для_грп,Дата ВНР после
2,1004,2011-02-04


In [32]:
step.loc[step['для_грп'] == '110']

,для_грп,Дата ВНР после


In [33]:
step = step[~step['для_грп'].isin(spisok)].reset_index(drop=True)
step.loc[step['для_грп'] == '110']

,для_грп,Дата ВНР после


In [34]:
vvv = step['для_грп'].tolist()
vvv

['1',
 '10',
 '1004',
 '1005',
 '1006',
 '105',
 '1092',
 '11',
 '1111',
 '1112',
 '1113',
 '1122',
 '1123',
 '1124',
 '1126',
 '1128',
 '1129',
 '119',
 '12',
 '122',
 '1220',
 '1221',
 '1223',
 '1224',
 '123',
 '1230Л',
 '123Л',
 '124',
 '1259',
 '1272',
 '128',
 '1291',
 '13',
 '130',
 '1302',
 '1304',
 '131',
 '1311',
 '1320',
 '133',
 '1334',
 '133Л',
 '1341',
 '1344',
 '1345',
 '1349',
 '1366',
 '1367',
 '1383',
 '1384',
 '1385',
 '1386',
 '139',
 '14',
 '141',
 '142Л',
 '143',
 '144',
 '146',
 '148',
 '149',
 '14Л',
 '15',
 '150',
 '1500',
 '1501Л',
 '1502',
 '1503',
 '1504',
 '1505',
 '1507',
 '1511',
 '1513',
 '151Л',
 '1521',
 '1523',
 '153',
 '154',
 '1544',
 '1549',
 '1549Л',
 '1550',
 '1551',
 '1555',
 '1556',
 '1556Л',
 '1557',
 '1559',
 '1560',
 '1561',
 '1562Л',
 '158',
 '16',
 '1602',
 '1605',
 '1606',
 '1607',
 '1608',
 '1609',
 '1610',
 '1611',
 '162',
 '1620',
 '1621',
 '1622',
 '1622Л',
 '1623',
 '1625',
 '1626',
 '1628',
 '1630',
 '1631',
 '1632',
 '1633',
 '1634'

In [35]:
step['Скважина №'] = vvv
step = step.drop(['для_грп'], axis=1)
step.head(10)

,Дата ВНР после,Скважина №
0,2016-03-12,1
1,2013-03-03,10
2,2011-02-04,1004
3,2011-08-31,1005
4,2019-08-16,1006
5,2010-10-23,105
6,2010-07-21,1092
7,2009-04-01,11
8,2015-03-27,1111
9,2015-05-07,1112


In [36]:
#добавим Л где нужно


In [37]:
dummy = dummy.merge(step, on=target, how='left')
dummy.head(10)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
0,2016-03-01,1,84.09,58.50,20.86,263.0,196.4,Ю3,131,2016-03-12
1,2016-04-01,1,71.61,47.27,24.90,263.0,172.5,Ю3,132,2016-03-12
2,2016-05-01,1,84.65,61.39,17.50,263.0,161.6,Ю3,133,2016-03-12
3,2016-06-01,1,79.36,60.95,12.62,263.0,149.5,Ю3,134,2016-03-12
4,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135,2016-03-12
5,2016-08-01,1,84.28,59.86,19.19,263.0,118.3,Ю3,136,2016-03-12
6,2016-09-01,1,85.31,68.98,8.01,263.0,107.1,Ю3,137,2016-03-12
7,2016-10-01,1,88.38,71.09,8.49,263.0,94.8,Ю3,138,2016-03-12
8,2016-11-01,1,78.99,62.62,9.81,263.0,79.4,Ю3,139,2016-03-12
9,2016-12-01,1,68.36,53.63,10.74,263.0,69.8,Ю3,140,2016-03-12


In [38]:
dummy.loc[dummy['Скважина №'] == '897Л']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после


In [39]:
dummy.loc[dummy['Скважина №'] == '110']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
210,2020-02-01,110,120.45,28.34,73.24,258.0,69.6,Ю3,9268,NaT
211,2020-03-01,110,84.35,31.32,57.76,258.0,69.6,Ю3,9269,NaT
212,2020-04-01,110,25.53,20.42,9.00,258.0,71.1,Ю3,9270,NaT
213,2020-05-01,110,15.08,12.08,8.85,258.0,42.5,Ю3,9271,NaT
214,2020-06-01,110,17.41,12.67,17.17,258.0,63.5,Ю3,9272,NaT
215,2020-07-01,110,17.46,11.88,22.57,258.0,60.9,Ю3,9273,NaT
216,2020-08-01,110,16.66,11.78,19.56,258.0,53.3,Ю3,9274,NaT
217,2020-09-01,110,16.96,10.76,27.82,258.0,52.3,Ю3,9275,NaT
218,2020-10-01,110,16.71,11.46,21.98,208.0,47.2,Ю3,9276,NaT
219,2020-11-01,110,15.69,10.72,22.32,208.0,61.9,Ю3,9277,NaT


In [40]:
dummy.loc[dummy['Скважина №'] == '1004']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
164,2011-02-01,1004,65.20,48.72,14.99,216.0,89.7,Ю3,878,2011-02-04
165,2011-03-01,1004,59.80,43.90,16.48,216.0,98.1,Ю3,879,2011-02-04
166,2011-04-01,1004,50.18,37.93,14.00,216.0,97.5,Ю3,880,2011-02-04
167,2011-05-01,1004,54.31,41.06,13.99,216.0,102.6,Ю3,881,2011-02-04
168,2011-06-01,1004,57.19,44.23,12.00,216.0,112.3,Ю3,882,2011-02-04
169,2011-07-01,1004,66.08,51.35,11.59,216.0,92.7,Ю3,883,2011-02-04
170,2011-08-01,1004,72.24,54.61,13.99,216.0,107.5,Ю3,884,2011-02-04
171,2011-09-01,1004,75.26,52.27,20.99,216.0,81.8,Ю3,885,2011-02-04
172,2011-10-01,1004,73.22,49.58,22.96,216.0,82.4,Ю3,886,2011-02-04
173,2011-11-01,1004,73.24,48.30,24.98,216.0,84.0,Ю3,887,2011-02-04


In [41]:
if dummy.shape[0] != 0:
    dummy[target] = dummy.apply(
    lambda x:
    str(x[target])
    + ('_ГРП' if x['Дата'] > x[date_column_name] else ''),
    axis=1
                )
if dummy.shape[0] != 0:
    dummy[target] = dummy.apply(
    lambda x:
    str(x[target])
    + ('_ГРП' if str(x['Дата'])[:7] in str( x[date_column_name]) else ''),
    axis=1
                )
dummy = dummy.drop(date_column_name, axis=1)
dummy.loc[dummy['Скважина №'] == '1004']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index


In [42]:
type(dummy['Дата'])

pandas.core.series.Series

In [43]:
dummy.loc[dummy['Скважина №'] == '1004_ГРП']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
164,2011-02-01,1004_ГРП,65.20,48.72,14.99,216.0,89.7,Ю3,878
165,2011-03-01,1004_ГРП,59.80,43.90,16.48,216.0,98.1,Ю3,879
166,2011-04-01,1004_ГРП,50.18,37.93,14.00,216.0,97.5,Ю3,880
167,2011-05-01,1004_ГРП,54.31,41.06,13.99,216.0,102.6,Ю3,881
168,2011-06-01,1004_ГРП,57.19,44.23,12.00,216.0,112.3,Ю3,882
169,2011-07-01,1004_ГРП,66.08,51.35,11.59,216.0,92.7,Ю3,883
170,2011-08-01,1004_ГРП,72.24,54.61,13.99,216.0,107.5,Ю3,884
171,2011-09-01,1004_ГРП,75.26,52.27,20.99,216.0,81.8,Ю3,885
172,2011-10-01,1004_ГРП,73.22,49.58,22.96,216.0,82.4,Ю3,886
173,2011-11-01,1004_ГРП,73.24,48.30,24.98,216.0,84.0,Ю3,887


In [44]:
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-03-01,1_ГРП,84.09,58.50,20.86,263.0,196.4,Ю3,131
1,2016-04-01,1_ГРП,71.61,47.27,24.90,263.0,172.5,Ю3,132
2,2016-05-01,1_ГРП,84.65,61.39,17.50,263.0,161.6,Ю3,133
3,2016-06-01,1_ГРП,79.36,60.95,12.62,263.0,149.5,Ю3,134
4,2016-07-01,1_ГРП,78.52,55.39,19.75,263.0,131.1,Ю3,135
...,...,...,...,...,...,...,...,...,...
22506,2019-08-01,96_ГРП,136.02,3.15,97.36,266.0,55.1,Ю3,308222
22507,2019-09-01,96_ГРП,114.20,4.17,95.85,266.0,72.9,Ю3,308223
22508,2019-10-01,96_ГРП,97.35,2.41,97.19,266.0,71.8,Ю3,308224
22509,2019-11-01,96_ГРП,94.11,2.59,96.87,266.0,65.5,Ю3,308225


In [45]:
dummy.to_excel('opa.xlsx')